# Manejo de bases de datos para las ciencias sociales II


## Manipulando la base de datos 

- Sorting
- Creación de columnas
- Casting
- Agregando datos
- Manejando valores missing 


A continuación veremos cuáles son los métodos más comunes para transformar nuestra base de datos y obtener nueva información. 

In [3]:
#importando pandas y leyendo nuestros datos

import pandas as pd


In [4]:
## Leyendo los archivos. 
locacion_datos = "aqui  coloca el string con la locación de tus datos"
locacion_datos = "/Users/ccsuehara/Downloads/Datos_abiertos_admision_2021_1_2024_1.csv" # este es el mio
df = pd.read_csv(locacion_datos)

In [ ]:
df.columns

In [ ]:
### Ordenar nuestra base de datos:
## una base de datos en no requiere que esté ordenada, pues la información mostrada es la misma,
# pero a veces es útil para visualizarla o para hacer operaciones.
df = df.sort_values('ANIO_NACIMIENTO', ascending = False) # por default, ascending es True. 
# lo mismo, pero inplace: 
#df.sort_values('ANIO_NACIMIENTO', ascending = False, inplace=True)

# ordenar por dos O más columnas: 
df = df.sort_values(by=['ANIO_NACIMIENTO', 'NACIMIENTO_PAIS'])

#df = df.sort_values(by=['ANIO_NACIMIENTO', 'NACIMIENTO_PAIS'], ascending=[False, True])



### Creando columnnas nuevas. 

A continuación veremos varias maneras de crear variables nuevas, dependiendo de las circunstancias de los datos, así como nuestras necesidades/preferencias.



In [8]:
# La manera más sencilla: 
# Crear una nueva columna a partir de la edad del postulante: 
df['MAYOR_EDAD'] = df['ANIO_NACIMIENTO'] <= 2006

Los siguentes métodos que utilizamos para filtrar datos también sirven para crear variables: 


`.loc `

Podemos crear nuevas variables ubicando en el dataframe los valores que cumplen cierta condición y reemplazando por otras. 

In [ ]:
df['SEXO_'] = 0
df.loc[df['SEXO'] == 'FEMENINO', 'SEXO_'] = 1

Si queremos cambiar varias categorías a la vez, podemos utilizar  `isin()`, `replace` o `where`: 

In [ ]:
paises = ['ARGENTINA', 'COLOMBIA', 'VENEZUELA', 'CHILE', 'ESPAÑA',
       'ITALIA', 'EE.UU.', 'BOLIVIA', 'BRAZIL', 'JAPÓN', 'KOREA',
       'URUGUAY', 'ECUADOR',  'GUATEMALA', 'HONDURAS',
       'SUIZA', 'SIRIA', 'PARAGUAY', 'FRANCIA', 'MÉXICO'] ## df['NACIMIENTO_PAIS'].unique()

df["extranjero"] = 0
df.loc[df['NACIMIENTO_PAIS'].isin(), "extranjero"] = 1

In [ ]:
df['OTRO_SEXO'] = df['SEXO']
df.replace({'OTRO_SEXO': {'FEMENINO': 1, 'MASCULINO': 0}}, inplace=True)

In [21]:
# import numpy as np
# df['extranjero'] = np.where(df['NACIMIENTO_PAIS'].isin(paises), 1, 0)


In [ ]:
df.columns

Utilizando `eval` podemos hacerlo con una sola expresión: 


In [ ]:
df['MAYOR_EDAD'] = df.eval('ANIO_NACIMIENTO <= 2006')
df['INGRESANTES_NOLIMA'] = df.eval("INGRESO == 'NO' & DOMICILIO_DEPA != 'LIMA'")



### Una forma más general: Utilizando apply y funciones lambda. 

Recuerdan las funciones labmda que vimos en la clase pasada? Estas resultan muy útiles para hacer operaciones en columnas de un dataframe cuando la vectorización no es posible. A continuación veremos algunos ejemplos: 

In [ ]:
df['MAYOR_EDAD_'] = df['ANIO_NACIMIENTO'].apply(lambda x: x <= 2006)


In [27]:
def mayor_edad(x):
    if x <= 2006:
        return True
    else:
        return False
    
df['MAYOR_EDAD_'] = df['ANIO_NACIMIENTO'].apply(lambda x: mayor_edad(x))

También podemos crear variables categóricas en base a variables contínuas con `cut`: 

In [38]:
categ_labels = ['cat_1', 'cat_2', 'cat_3', 'cat_4']
categ_bins = [-1, 5, 10, 15, 20.01]

In [ ]:
df['cat_calificacion'] = pd.cut(df['CALIF_FINAL'],
                              bins = categ_bins, labels = categ_labels)

### Casting: Cambiando tipos de datos: 
Hay veces cuando necetaremos cambiar el tipo de dato de una columna.    

- Por ejemplo, cuando el csv fue leido y una columna que debería ser numérica,  es leída como string.  
- Otra situación se da cuando queremos optimizar el uso de memoria de nuestra base de datos, y por ejemplo, queremos reducir el número de decimales que el sistema guarda.  
- Luego, cuando concatenamos bases de datos, los tipos entre uno y otro pueden no coincidir, y necesitaremos cambiarlos para poder hacer operaciones.


In [ ]:
df['ANIO_NACIMIENTO'] = df['ANIO_NACIMIENTO'].astype('str') 

# Cuando leemos los archivos, existe un argumento que se llama dtype, que nos permite especificar el tipo de dato de cada columna.


## Agregando datos


Para hacer agregaciones por columnas: ```axis = 0``` ó ```axis = 'index' ```
Para hacer agregaciones por filas: ```axis = 1``` ó ```axis = 'columns' ```

(dato curioso: El axis 0 en realidad se refiere a las filas, pero lo invocamos para obtener datos agregados por columnas porque es como lo mantenemos "fijo". Lo mismo para hacer operaciones por filas). 

| Función     | Descripción                                               |
|-------------|-----------------------------------------------------------|
| `mean`      | Calcula el promedio de los valores.                       |
| `median`    | Encuentra la mediana de los valores.                      |
| `sum`       | Suma todos los valores.                                   |
| `min`       | Encuentra el valor mínimo.                                |
| `max`       | Encuentra el valor máximo.                                |
| `quantile`  | Calcula el cuantil especificado de los valores.           |
| `std`       | Calcula la desviación estándar de los valores.            |
| `var`       | Calcula la varianza de los valores.                       |
| `count`     | Cuenta el número de valores no nulos.                     |


In [57]:
pbi = pd.read_csv("datos/pbi.csv")

departamentos = ['Amazonas', 'Áncash', 'Apurímac', 'Arequipa', 'Ayacucho',
       'Cajamarca', 'Cusco', 'Huancavelica', 'Huánuco', 'Ica', 'Junín',
       'La Libertad', 'Lambayeque', 'Lima', 'Prov. Const. del Callao',
       'Región Lima', 'Provincia de Lima', 'Loreto', 'Madre de Dios',
       'Moquegua', 'Pasco', 'Piura', 'Puno', 'San Martín', 'Tacna', 'Tumbes',
       'Ucayali']

In [ ]:
## Hallando la suma total de pbi departamental
pbi['total'] = pbi[departamentos].sum(axis=1)

## Hallando el promedio de pbi departamental de los últimos años:

pbi[departamentos].mean() # axis = 0 es por default. b

## Utilizando el groupby:

El groupby nos va a permitir agrupar los datos según una columna y aplicar una función a cada grupo. Es conveniente para cuando querramos obtener datos a un nivel más agregado. 

In [ ]:
df.columns

In [ ]:
df.groupby('ESPECIALIDAD')['CALIF_FINAL'].mean()
#.sort_values(ascending = False)

In [ ]:
df.groupby(['ESPECIALIDAD', 'INGRESO'])['CALIF_FINAL'].mean()
## qué pasa si colocas un .reset_index() al final? 


###  Una forma generalizada de hacer groupbys: 

In [ ]:

## aquí utilizamos el método agg para aplicar varias funciones a la vez. 
nueva_agregacion =df.groupby(['ESPECIALIDAD']).agg(
{'CALIF_FINAL': ['mean', 'max','min'], 
 'ANIO_NACIMIENTO': ['min', 'median']}
)

In [ ]:
## los nombres de las columnas son una lista de strings, que al tener varios niveles, tendrán un multiIndex. 
# En este curso no lidiaaremos con  multiIndex, así que te propongo la siguiente solución: 

nueva_agregacion.columns = [''.join(col).strip() for col in nueva_agregacion.columns.values]


In [ ]:
## Otra forma de hacerlo 

new_col = []
for col in nueva_agregacion.columns.values:
    a = ''.join(col).strip()
    new_col.append(a)
nueva_agregacion.columns = new_col


In [ ]:
## resumiendo datos con pivot_table()
Pandas tiene un método que nos permite hacer tablas dinámicas (a lo Excel) para resumir información. 



In [74]:
pd.pivot_table(df, values = 'CALIF_FINAL', index = 'ESPECIALIDAD', columns = 'INGRESO', aggfunc = 'mean')

INGRESO,NO,SI
ESPECIALIDAD,,
ARQUITECTURA,6.503551,13.034397
CIENCIA DE LA COMPUTACIÓN,7.069774,11.852496
FÍSICA,7.010372,12.053004
INGENIERÍA AMBIENTAL,5.955341,11.664649
INGENIERÍA CIVIL,6.817769,13.721998
INGENIERÍA DE CIBERSEGURIDAD,7.122163,12.833407
INGENIERÍA DE HIGIENE Y SEGURIDAD INDUSTRIAL,5.982545,11.258549
INGENIERÍA DE MINAS,6.249669,11.840015
INGENIERÍA DE PETRÓLEO Y GAS NATURAL,6.188209,12.195724


In [76]:
pd.pivot_table(df, values = ['CALIF_FINAL', 'ANIO_NACIMIENTO'],
                index = 'ESPECIALIDAD',
                  columns = 'INGRESO',
                    aggfunc = {'CALIF_FINAL': ['mean', 'median'], 
                               'ANIO_NACIMIENTO': 'median'})

ANIO_NACIMIENTO          \
                                                      median           
INGRESO                                                   NO      SI   
ESPECIALIDAD                                                           
ARQUITECTURA                                          2004.0  2003.0   
CIENCIA DE LA COMPUTACIÓN                             2004.0  2003.0   
FÍSICA                                                2004.0  2004.0   
INGENIERÍA AMBIENTAL                                  2004.0  2003.0   
INGENIERÍA CIVIL                                      2004.0  2003.0   
INGENIERÍA DE CIBERSEGURIDAD                          2005.0  2004.0   
INGENIERÍA DE HIGIENE Y SEGURIDAD INDUSTRIAL          2004.0  2002.0   
INGENIERÍA DE MINAS                                   2004.0  2004.0   
INGENIERÍA DE PETRÓLEO Y GAS NATURAL                  2003.0  2002.0   
INGENIERÍA DE SISTEMAS                                2005.0  2004.0   
INGENIERÍA DE SOFTWARE                                2006.0  2005.0   
INGENIERÍA DE TELECOMUNICACIONES                      2004.0  2003.0   
INGENIERÍA ECONÓMICA                                  2004.0  2003.0   
INGENIERÍA ELECTRÓNICA                                2004.0  2003.0   
INGENIERÍA ELÉCTRICA                                  2004.0  2003.0   
INGENIERÍA ESTADÍSTICA                                2005.0  2004.0   
INGENIERÍA FÍSICA                                     2004.0  2003.0   
INGENIERÍA GEOLÓGICA                                  2004.0  2003.0   
INGENIERÍA INDUSTRIAL                                 2004.0  2003.0   
INGENIERÍA MECATRÓNICA                                2004.0  2004.0   
INGENIERÍA MECÁNICA                                   2004.0  2003.0   
INGENIERÍA MECÁNICA-ELÉCTRICA                         2004.0  2003.0   
INGENIERÍA METALÚRGICA                                2004.0  2003.0   
INGENIERÍA NAVAL                                      2004.0  2003.0   
INGENIERÍA PETROQUÍMICA                               2004.0  2003.0   
INGENIERÍA QUÍMICA                                    2004.0  2004.0   
INGENIERÍA SANITARIA                                  2003.0  2002.0   
INGENIERÍA TEXTIL                                     2004.0  2002.0   
MATEMÁTICA                                            2004.0  2004.0   
QUÍMICA                                               2003.0  2003.0   

                                             CALIF_FINAL                     \
                                                    mean             median   
INGRESO                                               NO         SI      NO   
ESPECIALIDAD                                                                  
ARQUITECTURA                                    6.503551  13.034397  6.4105   
CIENCIA DE LA COMPUTACIÓN                       7.069774  11.852496  7.1660   
FÍSICA                                          7.010372  12.053004  7.0950   
INGENIERÍA AMBIENTAL                            5.955341  11.664649  5.9880   
INGENIERÍA CIVIL                                6.817769  13.721998  6.5000   
INGENIERÍA DE CIBERSEGURIDAD                    7.122163  12.833407  7.1085   
INGENIERÍA DE HIGIENE Y SEGURIDAD INDUSTRIAL    5.982545  11.258549  5.9500   
INGENIERÍA DE MINAS                             6.249669  11.840015  6.2220   
INGENIERÍA DE PETRÓLEO Y GAS NATURAL            6.188209  12.195724  5.8880   
INGENIERÍA DE SISTEMAS                          6.309844  13.354662  5.9735   
INGENIERÍA DE SOFTWARE                          6.947527  13.778621  6.8400   
INGENIERÍA DE TELECOMUNICACIONES                7.158423  13.012861  7.3060   
INGENIERÍA ECONÓMICA                            6.381207  11.592202  6.2945   
INGENIERÍA ELECTRÓNICA                          7.217361  13.006515  7.2840   
INGENIERÍA ELÉCTRICA                            6.973840  12.869580  7.0240   
INGENIERÍA ESTADÍSTICA                          6.598870  11.396682  6.4790   
INGENIERÍA FÍSIC

### Manejo de datos faltantes:
Los datos faltantes pueden suceder por varias razones:
-  Cuando no se necesita registrar los datos de dicha observación (por ejemplo, nivel educativo para menores de 3 años).  
-  Sin embargo, esto puede afectar nuestro análisis si son errores en el levantamiento de la información.    
    - ejemplo de variables sensibles de levantamiento. 
    - errores de levantamiento de datos. 
    
Su identifcación y tratamiento es importante. 


In [ ]:
## IDentificando datos faltantes:
df.isna().sum() # nos da la cantidad de datos faltantes por columna.

In [ ]:
## Observando datos faltantes de un subconjunto de columnas:
df[['COLEGIO_DEPA', 'NACIMIENTO_DEPA']].isna().sum()
